# Predictive Analtics Exam 2

### 15. Considering the train.csv and test.csv data files containing information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. The goal is to predict default payment next month on the test.csv data file

In [1]:
## Importing necessary libraries

import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import precision_recall_cutoff as prc
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [2]:
## a) Using the pandas library to read the train.csv and test.csv data files and create two data-frames called train and test

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

file_key = 'train(1).csv'
file_key2 = 'test(1).csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')

train = pd.read_csv(file_content_stream)
test = pd.read_csv(file_content_stream2)

train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,400000,1,1,2,32,0,0,0,0,0,0,55773,55917,51389,48272,49478,51242,3028,3023,3000,3000,3000,38662,0
1,120000,2,2,2,30,-1,-1,-1,-1,-1,-1,140,3230,3011,1964,1883,1538,3230,3011,1964,1883,1538,1911,0
2,270000,2,2,2,32,0,0,0,0,0,0,59710,49986,104390,94856,86461,83650,1808,69563,2891,2689,3012,2771,0
3,280000,2,2,1,27,0,0,0,0,0,0,280913,283222,273160,257689,193231,191143,11052,9563,15017,5374,5420,6021,0
4,30000,2,1,2,27,0,0,-1,0,0,-2,1512,2458,664,1814,0,0,1000,664,1500,0,0,0,0


In [3]:
## b) Splitting the train data-frame intro training (80%) and validation (20%) (taking into account the proportions of 0s and 1s)

## Defining the input and target variables
X = train.drop(columns = ['default payment next month'])
Y = train['default payment next month']

## Splitting the data
X_training, X_validation, Y_training, Y_validation = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [ ]:
## Engineering features from Exam 1


## Training set:

## Most common repayment status
X_training['Most_Common'] = np.nan
for i in range(0, X_training.shape[0]):
    X_training.at[i, 'Most_Common'] = X_training[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].loc[i].mode()[0]

## From plot tree:
X_training['Tree2'] = np.where((X_training['PAY_0'] <= 1.5) & (X_training['PAY_2'] <= 1.5) & (X_training['PAY_AMT3'] > 395.0), 1, 0)
X_training['Tree6'] = np.where((X_training['PAY_0'] > 1.5) & (X_training['PAY_6'] <= 1.0) & (X_training['BILL_AMT1'] > 649.5), 1, 0)
X_training['Tree7'] = np.where((X_training['PAY_0'] > 1.5) & (X_training['PAY_6'] > 1.0) & (X_training['PAY_AMT3'] <= 14177.0), 1, 0)



## Validation set:

## Most common repayment status
X_validation['Most_Common'] = np.nan
for i in range(0, X_validation.shape[0]):
    X_validation.at[i, 'Most_Common'] = X_validation[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].loc[i].mode()[0]

## From plot tree:
X_validation['Tree2'] = np.where((X_validation['PAY_0'] <= 1.5) & (X_validation['PAY_2'] <= 1.5) & (X_validation['PAY_AMT3'] > 395.0), 1, 0)
X_validation['Tree6'] = np.where((X_validation['PAY_0'] > 1.5) & (X_validation['PAY_6'] <= 1.0) & (X_validation['BILL_AMT1'] > 649.5), 1, 0)
X_validation['Tree7'] = np.where((X_validation['PAY_0'] > 1.5) & (X_validation['PAY_6'] > 1.0) & (X_validation['PAY_AMT3'] <= 14177.0), 1, 0)



## Test set:

## Most common repayment status
test['Most_Common'] = np.nan
for i in range(0, test.shape[0]):
    test.at[i, 'Most_Common'] = test[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].loc[i].mode()[0]

## From plot tree:
test['Tree2'] = np.where((test['PAY_0'] <= 1.5) & (test['PAY_2'] <= 1.5) & (test['PAY_AMT3'] > 395.0), 1, 0)
test['Tree6'] = np.where((test['PAY_0'] > 1.5) & (test['PAY_6'] <= 1.0) & (test['BILL_AMT1'] > 649.5), 1, 0)
test['Tree7'] = np.where((test['PAY_0'] > 1.5) & (test['PAY_6'] > 1.0) & (test['PAY_AMT3'] <= 14177.0), 1, 0)

In [ ]:
## c) Using the top 7 variables from Exercise 15 part (e) in Exam 1 to  build a model on the training data-frame, and tuning
## this model on the validation data-frame


## Redefining the input and target variables
X_training = training[['PAY_0', 'PAY_2', 'Most_Common', 'Tree2', 'Tree6', 'PAY_3', 'Tree7']]
Y_training = training['default payment next month']

X_testing = testing[['PAY_0', 'PAY_2', 'Most_Common', 'Tree2', 'Tree6', 'PAY_3', 'Tree7']]
Y_testing = testing['default payment next month']





Use the provided precision recall cutoff.py (posted under the Exam
2 link) file to estimate the optimal cutoff value. Report the F1-score of the model.
Finally, use the optimal model to predict the likelihood of default payment next
month on the test.